In [ ]:
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt

import pickle

from qubic.lib.QskySim import get_angular_profile, get_noise_invcov_profile

from pysimulators.interfaces.healpy import HealpixConvolutionGaussianOperator

In [ ]:
pickle_file_w = pickle.load(open("test_edge_effect/maps/CMB_w_convo_w_Planck_noiseless_test_None.pkl", "rb"))
pickle_file_wo = pickle.load(open("test_edge_effect/maps/CMB_w_convo_w_Planck_noiseless.pkl", "rb"))

In [ ]:
pickle_file_w['parameters']

In [ ]:
hp.gnomview((pickle_file_w['maps_noise'] - (pickle_file_w["maps"] - pickle_file_w['maps_in'][:1]))[0, :, 0], 
            rot=pickle_file_w['center'])

In [ ]:
maps_in_w = pickle_file_w['maps_in']
maps_rec_w = pickle_file_w['maps']
residuals_w = pickle_file_w['maps_noise']

maps_in_wo = pickle_file_wo['maps_in']
maps_rec_wo = pickle_file_wo['maps']
residuals_wo = pickle_file_wo['maps_noise']

In [ ]:
print(pickle_file_w['fwhm_in'])
print(pickle_file_w['fwhm_rec'])

In [ ]:
# With convolutions
plt.figure(figsize=(10, 12))
k=1

stk = ['I', 'Q', 'U']
inu = 0
istk = 0
n_sig = 3
reso = 15

C = HealpixConvolutionGaussianOperator(pickle_file_w['fwhm_rec'][inu]*0)
sigma = np.std(C*maps_in_w[inu, :, istk])

hp.gnomview(C*maps_in_w[inu, :, istk], cmap='jet', rot=pickle_file_w['center'], reso=reso, sub=(1, 3, k), title=f'{stk[istk]} - Input', notext=True)
hp.gnomview(maps_rec_w[inu, :, istk], cmap='jet', rot=pickle_file_w['center'], reso=reso, sub=(1, 3, k+1), title=f'{stk[istk]} - Output', notext=True)
hp.gnomview(maps_rec_w[inu, :, istk] - C*maps_in_w[inu, :, istk], cmap='jet', rot=pickle_file_w['center'], reso=reso, sub=(1, 3, k+2), title=f'{stk[istk]} - Residual', notext=True)
k+=3

In [ ]:
# Without convolutions
plt.figure(figsize=(10, 12))
k=1

stk = ['I', 'Q', 'U']
inu = 0
istk = 0  
n_sig = 3
reso = 15

C = HealpixConvolutionGaussianOperator(pickle_file_wo['fwhm_rec'][inu]*0)
sigma = np.std(C*maps_in_wo[inu, :, istk])

hp.gnomview(C*maps_in_wo[inu, :, istk], cmap='jet', rot=pickle_file_wo['center'], reso=reso, sub=(1, 3, k), title=f'{stk[istk]} - Input', notext=True)
hp.gnomview(maps_rec_wo[inu, :, istk], cmap='jet', rot=pickle_file_wo['center'], reso=reso, sub=(1, 3, k+1), title=f'{stk[istk]} - Output', notext=True)
hp.gnomview(maps_rec_wo[inu, :, istk] - C*maps_in_wo[inu, :, istk], cmap='jet', rot=pickle_file_wo['center'], reso=reso, sub=(1, 3, k+2), title=f'{stk[istk]} - Residual', notext=True)
k+=3

In [ ]:
tod_w = pickle_file_w['tod']
tod_wo = pickle_file_wo['tod']

In [ ]:
convergence_w = pickle_file_w['convergence']
convergenc_wo = pickle_file_wo['convergence']
plt.plot(convergence_w, label='With Convolution')
plt.plot(convergenc_wo, label='Without Convolution')
plt.xlabel('iterations')
plt.ylabel('normalized residual')
plt.title('Convergence of the PCG algorithm')
plt.yscale('log')
plt.legend()
plt.show()

In [ ]:
plt.plot(tod_w, label='With Convolution', alpha=0.5)
plt.plot(tod_wo, label='Without Convolution', alpha=0.5)
plt.title('TOD')
plt.legend()

In [ ]:
coverage_w = pickle_file_w["coverage"]
test_w = np.ones((coverage_w.size, 3)) * coverage_w[:, None]
get_angular_profile(test_w, doplot=True, nbins=100, label="with convolution")

coverage_wo = pickle_file_wo["coverage"]
test_wo = np.ones((coverage_wo.size, 3)) * coverage_wo[:, None]
get_angular_profile(test_wo, doplot=True, nbins=100, label="without convolution")

In [ ]:
get_angular_profile(residuals_w[0], doplot=True, allstokes=True, nbins=80, thmax=20, label="With convolution")
plt.figure()
get_angular_profile(residuals_wo[0], doplot=True, allstokes=True, nbins=80, thmax=20, label="Without convolution")
plt.xlim(0, 2.5)
plt.ylim(0, 0.001)

In [ ]:
get_noise_invcov_profile(residuals_w[0], coverage_w, doplot=True, allstokes=True, label="With Convolution")
plt.figure()
get_noise_invcov_profile(residuals_wo[0], coverage_wo, doplot=True, allstokes=True, label="Without Convolution")